[View in Colaboratory](https://colab.research.google.com/github/aminzabardast/Tensorflow-Tutorials/blob/dev/4_Visualization_Using_Tensorboard.ipynb)

# Visualization Using Tensorboard

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
np.random.seed(seed=23)
xs = np.linspace(-1, 1, 300)[:, np.newaxis]
noise = np.random.normal(0, .1, size=xs.shape)
ys = np.square(xs) - 0.5 + noise

In [0]:
def add_layer(input, in_size, out_size, name, activation_function=None, seed=None):
    with tf.name_scope('Layer_%s' % name):
        with tf.name_scope('Weights'):
            weights = tf.Variable(tf.random_normal([in_size, out_size], seed=seed))
        with tf.name_scope('Biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
        wx_plus_b = tf.matmul(input, weights) + biases
        output = activation_function(wx_plus_b) if activation_function else wx_plus_b
        return output

In [0]:
with tf.name_scope('Input'):
    xp = tf.placeholder(tf.float32, shape=[None, 1], name='InputVector')
with tf.name_scope('Truth'):
    yp = tf.placeholder(tf.float32, shape=[None, 1], name='LabelVector')

In [0]:
l1 = add_layer(xp, 1, 3, '1', tf.nn.relu, seed=2)
l2 = add_layer(l1, 3, 10, '2', tf.nn.relu, seed=2)
l3 = add_layer(l2, 10, 3, '3', tf.nn.relu, seed=2)
prediction = add_layer(l3, 3, 1, 'Output', seed=2)

In [0]:
with tf.name_scope('Loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(yp-prediction), reduction_indices=[1]))
    tf.summary.scalar(name='MSE', tensor=loss)

In [0]:
with tf.name_scope('Train'):
    train_step = tf.train.GradientDescentOptimizer(learning_rate=.1).minimize(loss=loss)

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()

In [0]:
merged = tf.summary.merge_all(key=tf.GraphKeys.SUMMARIES)
writer = tf.summary.FileWriter(logdir='logs/', graph=sess.graph)
sess.run(init)

#### **Colaboratory Note:**
Do not run previous steps multiple times. This breaks the session. If needed, **restart the run time** from `Runtime > Restart runtime...` .

In [0]:
for i in range(10000):
  sess.run(train_step, feed_dict={xp: xs, yp: ys})
  result = sess.run(merged, feed_dict={xp: np.array(xs), yp: np.array(ys)})
  writer.add_summary(result, i)

In [0]:
!ls

In [0]:
!tar czvf logs.tar.gz logs
!ls

In [0]:
from google.colab import files
files.download('logs.tar.gz')

In [0]:
!rm -rvf logs*
!ls